Простой пример: Пульт управления светом
Имитируем простой пульт с одной кнопкой, которая может выполнять разные команды (включить/выключить свет).

In [1]:
from abc import ABC, abstractmethod

# 1. Receiver (Получатель)
class Light:
    """Объект, который будет выполнять действия (Получатель)."""
    def __init__(self, location: str = ""):
        self.location = location
        self._is_on = False

    def turn_on(self):
        if not self._is_on:
            print(f"Light in {self.location}: Turning ON")
            self._is_on = True
        else:
            print(f"Light in {self.location}: Already ON")

    def turn_off(self):
        if self._is_on:
            print(f"Light in {self.location}: Turning OFF")
            self._is_on = False
        else:
            print(f"Light in {self.location}: Already OFF")

# 2. Command Interface (Интерфейс Команды)
class Command(ABC):
    """Интерфейс для всех команд."""
    @abstractmethod
    def execute(self) -> None:
        pass

# 3. Concrete Commands (Конкретные Команды)
class LightOnCommand(Command):
    """Команда для включения света."""
    def __init__(self, light: Light):
        self._light = light # Хранит ссылку на Получателя

    def execute(self) -> None:
        print(f"LightOnCommand: Executing...")
        self._light.turn_on() # Вызывает метод Получателя

class LightOffCommand(Command):
    """Команда для выключения света."""
    def __init__(self, light: Light):
        self._light = light

    def execute(self) -> None:
        print(f"LightOffCommand: Executing...")
        self._light.turn_off()

# 4. Invoker (Инициатор)
class SimpleRemoteControl:
    """Простой пульт с одной кнопкой (Инициатор)."""
    def __init__(self):
        self._slot: Command | None = None # Место для команды

    def set_command(self, command: Command):
        """Назначает команду кнопке."""
        print(f"RemoteControl: Setting command to {type(command).__name__}")
        self._slot = command

    def button_was_pressed(self):
        """Выполняет назначенную команду."""
        print("RemoteControl: Button pressed!")
        if self._slot:
            self._slot.execute() # Вызывает execute() у команды
        else:
            print("RemoteControl: No command assigned to the slot.")

# 5. Client Code (Клиентский Код)
if __name__ == "__main__":
    # Создаем Получателя
    living_room_light = Light("Living Room")

    # Создаем Команды, связывая их с Получателем
    light_on = LightOnCommand(living_room_light)
    light_off = LightOffCommand(living_room_light)

    # Создаем Инициатора
    remote = SimpleRemoteControl()

    # Конфигурируем Инициатора: назначаем команду включения
    remote.set_command(light_on)
    # Нажимаем кнопку -> выполняется команда включения
    remote.button_was_pressed()
    remote.button_was_pressed() # Повторное нажатие

    print("-" * 20)

    # Переназначаем команду на выключение
    remote.set_command(light_off)
    # Нажимаем кнопку -> выполняется команда выключения
    remote.button_was_pressed()
    remote.button_was_pressed() # Повторное нажатие

# Вывод:
# RemoteControl: Setting command to LightOnCommand
# RemoteControl: Button pressed!
# LightOnCommand: Executing...
# Light in Living Room: Turning ON
# RemoteControl: Button pressed!
# LightOnCommand: Executing...
# Light in Living Room: Already ON
# --------------------
# RemoteControl: Setting command to LightOffCommand
# RemoteControl: Button pressed!
# LightOffCommand: Executing...
# Light in Living Room: Turning OFF
# RemoteControl: Button pressed!
# LightOffCommand: Executing...
# Light in Living Room: Already OFF

RemoteControl: Setting command to LightOnCommand
RemoteControl: Button pressed!
LightOnCommand: Executing...
Light in Living Room: Turning ON
RemoteControl: Button pressed!
LightOnCommand: Executing...
Light in Living Room: Already ON
--------------------
RemoteControl: Setting command to LightOffCommand
RemoteControl: Button pressed!
LightOffCommand: Executing...
Light in Living Room: Turning OFF
RemoteControl: Button pressed!
LightOffCommand: Executing...
Light in Living Room: Already OFF


Сложный пример: Текстовый Редактор с Undo/Redo
Реализуем простейший текстовый редактор, где команды вставки и удаления текста можно отменять.

In [2]:
from abc import ABC, abstractmethod
from collections import deque # Используем deque для истории

# 1. Receiver (Получатель)
class TextEditor:
    """Получатель: содержит текст и методы для его изменения."""
    def __init__(self):
        self._content = ""
        print("TextEditor initialized.")

    def insert_text(self, position: int, text: str):
        """Вставляет текст в указанную позицию."""
        print(f"EDITOR: Inserting '{text}' at position {position}")
        self._content = self._content[:position] + text + self._content[position:]
        print(f"EDITOR: Current content: '{self._content}'")

    def delete_text(self, position: int, length: int) -> str:
        """Удаляет текст и возвращает удаленный фрагмент (важно для undo)."""
        if position < 0 or position >= len(self._content) or length <= 0:
            print(f"EDITOR: Invalid delete operation at {position} for length {length}")
            return ""
        end_position = min(position + length, len(self._content))
        deleted_text = self._content[position:end_position]
        print(f"EDITOR: Deleting '{deleted_text}' from position {position} (length {length})")
        self._content = self._content[:position] + self._content[end_position:]
        print(f"EDITOR: Current content: '{self._content}'")
        return deleted_text

    def get_content(self) -> str:
        return self._content

# 2. Command Interface (Интерфейс Команды с Undo)
class Command(ABC):
    """Интерфейс команды, теперь с методом undo."""
    def __init__(self, editor: TextEditor):
        self._editor = editor
        self._backup = "" # Для хранения состояния для undo

    @abstractmethod
    def execute(self) -> None:
        pass

    @abstractmethod
    def undo(self) -> None:
        pass

# 3. Concrete Commands (Конкретные Команды с Undo)
class InsertCommand(Command):
    """Команда вставки текста."""
    def __init__(self, editor: TextEditor, position: int, text: str):
        super().__init__(editor)
        self._position = position
        self._text = text
        # Для undo нам нужно знать позицию и длину вставленного текста
        self._inserted_length = len(text)

    def execute(self) -> None:
        print(f"InsertCommand: Executing insert '{self._text}' at {self._position}")
        # Важно: backup не нужен, т.к. undo удаляет то, что мы вставили
        self._editor.insert_text(self._position, self._text)

    def undo(self) -> None:
        print(f"InsertCommand: Undoing insert of '{self._text}' at {self._position}")
        # Для отмены вставки нужно удалить вставленный текст
        self._editor.delete_text(self._position, self._inserted_length)

class DeleteCommand(Command):
    """Команда удаления текста."""
    def __init__(self, editor: TextEditor, position: int, length: int):
        super().__init__(editor)
        self._position = position
        self._length = length
        self._deleted_text: str | None = None # Храним удаленный текст для undo

    def execute(self) -> None:
        print(f"DeleteCommand: Executing delete at {self._position} for length {self._length}")
        # Выполняем удаление и сохраняем результат для undo
        self._deleted_text = self._editor.delete_text(self._position, self._length)

    def undo(self) -> None:
        if self._deleted_text is not None:
            print(f"DeleteCommand: Undoing delete by re-inserting '{self._deleted_text}' at {self._position}")
            # Для отмены удаления нужно вставить удаленный текст обратно
            self._editor.insert_text(self._position, self._deleted_text)
        else:
            print("DeleteCommand: Cannot undo, no deleted text was saved.")


# 4. Invoker (Инициатор с историей для Undo)
class EditorInvoker:
    """Инициатор, который хранит историю команд для undo."""
    def __init__(self):
        self._history = deque() # Используем deque как стек
        print("EditorInvoker initialized with history.")

    def execute_command(self, command: Command):
        """Выполняет команду и добавляет её в историю."""
        print(f"\nINVOKER: Executing command {type(command).__name__}...")
        command.execute()
        self._history.append(command)
        print(f"INVOKER: Command {type(command).__name__} added to history. History size: {len(self._history)}")

    def undo_last_command(self):
        """Отменяет последнюю выполненную команду."""
        if not self._history:
            print("\nINVOKER: History is empty, nothing to undo.")
            return

        last_command = self._history.pop()
        print(f"\nINVOKER: Undoing command {type(last_command).__name__}...")
        last_command.undo()
        print(f"INVOKER: Undo complete. History size: {len(self._history)}")

# 5. Client Code
if __name__ == "__main__":
    # Создаем Получателя и Инициатора
    editor = TextEditor()
    invoker = EditorInvoker()

    # Клиент создает и выполняет команды через Инициатора
    cmd1 = InsertCommand(editor, 0, "Hello")
    invoker.execute_command(cmd1)

    cmd2 = InsertCommand(editor, 5, " World")
    invoker.execute_command(cmd2)

    cmd3 = DeleteCommand(editor, 5, 6) # Удаляем " World"
    invoker.execute_command(cmd3)

    cmd4 = InsertCommand(editor, 5, " Python!")
    invoker.execute_command(cmd4)

    print(f"\nFINAL CONTENT: '{editor.get_content()}'")

    # Отменяем последние действия
    invoker.undo_last_command() # Отменяем вставку " Python!"
    invoker.undo_last_command() # Отменяем удаление " World"
    invoker.undo_last_command() # Отменяем вставку " World"
    invoker.undo_last_command() # Отменяем вставку "Hello"
    invoker.undo_last_command() # Пытаемся отменить еще раз (история пуста)

    print(f"\nCONTENT AFTER UNDO: '{editor.get_content()}'")

# Примерный вывод:
# TextEditor initialized.
# EditorInvoker initialized with history.
#
# INVOKER: Executing command InsertCommand...
# InsertCommand: Executing insert 'Hello' at 0
# EDITOR: Inserting 'Hello' at position 0
# EDITOR: Current content: 'Hello'
# INVOKER: Command InsertCommand added to history. History size: 1
#
# INVOKER: Executing command InsertCommand...
# InsertCommand: Executing insert ' World' at 5
# EDITOR: Inserting ' World' at position 5
# EDITOR: Current content: 'Hello World'
# INVOKER: Command InsertCommand added to history. History size: 2
#
# INVOKER: Executing command DeleteCommand...
# DeleteCommand: Executing delete at 5 for length 6
# EDITOR: Deleting ' World' from position 5 (length 6)
# EDITOR: Current content: 'Hello'
# INVOKER: Command DeleteCommand added to history. History size: 3
#
# INVOKER: Executing command InsertCommand...
# InsertCommand: Executing insert ' Python!' at 5
# EDITOR: Inserting ' Python!' at position 5
# EDITOR: Current content: 'Hello Python!'
# INVOKER: Command InsertCommand added to history. History size: 4
#
# FINAL CONTENT: 'Hello Python!'
#
# INVOKER: Undoing command InsertCommand...
# InsertCommand: Undoing insert of ' Python!' at 5
# EDITOR: Deleting ' Python!' from position 5 (length 8)
# EDITOR: Current content: 'Hello'
# INVOKER: Undo complete. History size: 3
#
# INVOKER: Undoing command DeleteCommand...
# DeleteCommand: Undoing delete by re-inserting ' World' at 5
# EDITOR: Inserting ' World' at position 5
# EDITOR: Current content: 'Hello World'
# INVOKER: Undo complete. History size: 2
#
# INVOKER: Undoing command InsertCommand...
# InsertCommand: Undoing insert of ' World' at 5
# EDITOR: Deleting ' World' from position 5 (length 6)
# EDITOR: Current content: 'Hello'
# INVOKER: Undo complete. History size: 1
#
# INVOKER: Undoing command InsertCommand...
# InsertCommand: Undoing insert of 'Hello' at 0
# EDITOR: Deleting 'Hello' from position 0 (length 5)
# EDITOR: Current content: ''
# INVOKER: Undo complete. History size: 0
#
# INVOKER: History is empty, nothing to undo.
#
# CONTENT AFTER UNDO: ''

TextEditor initialized.
EditorInvoker initialized with history.

INVOKER: Executing command InsertCommand...
InsertCommand: Executing insert 'Hello' at 0
EDITOR: Inserting 'Hello' at position 0
EDITOR: Current content: 'Hello'
INVOKER: Command InsertCommand added to history. History size: 1

INVOKER: Executing command InsertCommand...
InsertCommand: Executing insert ' World' at 5
EDITOR: Inserting ' World' at position 5
EDITOR: Current content: 'Hello World'
INVOKER: Command InsertCommand added to history. History size: 2

INVOKER: Executing command DeleteCommand...
DeleteCommand: Executing delete at 5 for length 6
EDITOR: Deleting ' World' from position 5 (length 6)
EDITOR: Current content: 'Hello'
INVOKER: Command DeleteCommand added to history. History size: 3

INVOKER: Executing command InsertCommand...
InsertCommand: Executing insert ' Python!' at 5
EDITOR: Inserting ' Python!' at position 5
EDITOR: Current content: 'Hello Python!'
INVOKER: Command InsertCommand added to history. 